In [10]:
from ray.rllib.offline.json_reader import JsonReader
import numpy as np
import numpy_indexed as npi

Find target sequence in dataset

View next state from sequence

In [61]:
input_reader = JsonReader("logs/APPO/RE3")
sequence_length=10
num_episodes = 9000
windows_per_episode = 100-sequence_length
state_len = 52
num_action = 145
encoding_len = state_len + num_action
state_to_index = {}
index_to_state = {}
state_index = 0

data_points = num_episodes * windows_per_episode

X = np.zeros((data_points,sequence_length,encoding_len))
Y_List = []

index = 0
for e in range(num_episodes):
    data = input_reader.next()
    data['obs'].shape
    for i in range(0, windows_per_episode):
        if not data['obs'][i+1].tobytes() in state_to_index:
            state_to_index[data['obs'][i+1].tobytes()] = state_index
            index_to_state[state_index] = data['obs'][i+1].tobytes()
            state_index += 1
        ns = state_to_index[data['obs'][i+1].tobytes()]
        if i < sequence_length:
            s = np.zeros((sequence_length, encoding_len))
            for k in range(0,i):
                s[k,:] = np.concatenate([data['obs'][k], np.zeros(num_action)])
        else:
            vec = np.zeros((sequence_length, num_action))
            vec[np.arange(sequence_length),data['actions'][i-sequence_length:i]] = 1
            s = np.concatenate([data['obs'][i-sequence_length:i], vec], axis=1)
            
        X[index,:,:] = s
        Y_List.append(ns)
        index += 1

2022-12-16 17:16:46,935	WARNING json_reader.py:238 -- Treating input directory as glob patterns: ['/tf/Cage/Notebooks/logs/APPO/RE3/*.json', '/tf/Cage/Notebooks/logs/APPO/RE3/*.zip']


In [81]:
new_state = {}

target = np.load('target.npy')

for i, matrix in enumerate(X):
    if (X[3]==matrix).all():
        print(i)
        if not Y_List[i] in new_state:
            new_state[Y_List[i]] = 1
        else:
            new_state[Y_List[i]] += 1
            
for s in new_state.keys():
    print(new_state[s])
    print(np.frombuffer(index_to_state[s]))

print(len(new_state.keys()))

3
303123
568083
643863
3
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.
 0. 0. 0. 0.]
1
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.
 1. 0. 0. 0.]
2


In [72]:
matrix.shape

(10, 197)

In [66]:
np.load('prediction.npy')

array([3.4389514e-02, 3.5582154e-10, 3.0057395e-10, 3.1300681e-10,
       1.3044986e-01, 2.3578420e-10, 8.6121874e-09, 6.8961858e-09,
       5.9871059e-02, 7.0032543e-03, 2.2421193e-05, 9.8166978e-01,
       5.8809634e-02, 2.5626041e-06, 2.8129612e-05, 3.0783215e-03,
       1.4888089e-03, 2.1715825e-10, 4.0353715e-10, 2.9277455e-10,
       6.4575978e-02, 2.4832056e-10, 2.9085487e-10, 2.9280808e-10,
       8.3733047e-04, 2.8988456e-10, 2.6997030e-10, 2.4731472e-10,
       2.9275443e-02, 1.4144541e-07, 4.8113784e-06, 1.1242133e-04,
       4.3243751e-02, 2.3672372e-10, 2.5623439e-10, 2.5736485e-10,
       7.8579538e-02, 2.6089807e-05, 7.9790680e-03, 9.9998879e-01,
       9.5541716e-02, 7.8056950e-11, 3.8660493e-09, 3.2103421e-06,
       2.0949865e-02, 5.5601357e-10, 2.2187642e-13, 2.2611125e-07,
       3.7523702e-02, 1.3491926e-07, 4.0378370e-13, 2.3167106e-06],
      dtype=float32)

In [42]:
new_state_c = {}

prediction = np.load('prediction.npy')
for i in range(1000):
    state = np.zeros(52)
    for i in range(52):
        state[i] = np.array(np.random.rand() < prediction[i], dtype=np.int8)
    if not state.tobytes() in new_state_c:
        new_state_c[state.tobytes()] = 1
    else:
        new_state_c[state.tobytes()] += 1
            
for s in new_state_c.keys():
    print(new_state_c[s])
    print(np.frombuffer(s))

print(len(new_state_c.keys()))

487
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0.]
37
[0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0.]
20
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0.]
29
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0.]
52
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0.]
12
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0.]
29
[0. 0. 0. 0. 0. 0.